In [10]:
#!pip install dash-bootstrap-components
#!pip install jupyter-dash

In [2]:
from jupyter_dash import JupyterDash 
from dash import dcc
from dash import html
from dash.dependencies import Input, Output 
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
from imp import reload
#import pandas_datareader.data as web #it is an in-built pandas API to read data from the web
#import datetime

# change 1

import plotting modules this way

In [3]:

from plot_functions import median_profiles, plot_times_series
reload(median_profiles)
reload(plot_times_series)

<module 'plot_functions.plot_times_series' from 'c:\\Users\\PC_victor\\Documents\\6_side_projects\\battery_dev\\BatteryDevBBoxx\\notebooks\\report\\plot_functions\\plot_times_series.py'>

# Load data

data is for now stored on my local machine. need a script to load and generate it from raw data

In [ ]:
%run 0_download_data.ipynb

In [3]:
%run 1_clean_data_backbone.ipynb

  0%|          | 0/4 [00:00<?, ?it/s]

Loading data...


 25%|██▌       | 1/4 [00:11<00:35, 11.72s/it]

Loading data...


 50%|█████     | 2/4 [00:20<00:19,  9.96s/it]

Loading data...


 75%|███████▌  | 3/4 [00:28<00:09,  9.11s/it]

Loading data...


100%|██████████| 4/4 [00:36<00:00,  9.25s/it]


Finished...
Concatenate...
Finished...
Doesn't exists


In [4]:
clean_files = ['../data/cleaned_data/all_dataset.parquet']
dfs = []
for file in clean_files:
    dfs.append(pd.read_parquet(file))
df = pd.concat(dfs, axis = 0)

df.index = pd.to_datetime(df.index)
df['state_of_charge_percent'] = df['state_of_charge_percent'].clip(0,120)

In [5]:
df['month_name'] = df.index.month_name()
list_months = ['Jan', 'Mar', 'Jun']
option_resampling = [{'label':'month','value':'1MS'},{'label':'day','value':'1D'}]

In [6]:
list_months = ['Jan', 'Mar', 'Jun']
options_fields = [{'label': c, 'value': c} for c in df.columns]
options_n_random_batteries = [{'label':f'{c} random batteries','value': c} for c in range(1, df.battery_id.nunique(), 5)]

# Full dash app

In [10]:
#Building the app
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.SKETCHY],
                            meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]) #if you want to make it adjust when the dashboard is visualized on mobile phones

#Building the layout
app.layout = dbc.Container([
                            #Row 1: title + subtitle
                            dbc.Row([
                                     dbc.Col([html.H1('BBoxx Lithium-Ion Battery Field Data Challenge', className = 'text-center mb-4')], width = 12),
                                     dbc.Col([html.P('Team 22: Victor Bossard, Etienne Beauchamp, Elias Galiounas, Maria Varini', className = 'text-center mb-4')], width = 12),
                            ]), #close Row 1

                            #Row 2: Data visualization
                            dbc.Row([
                                     #Title of the section
                                        dbc.Col([html.H1('Data visualization', className = 'text-primary text-left mb-4')], width = 12),
                                        #Column 1 (left): Dropdown list + time-series graph
                                        dbc.Col([dcc.Dropdown(id = 'n_batteries_ts-dropdown',
                                                                value = 5,
                                                                options = options_n_random_batteries),
                                                dcc.Dropdown(id = 'var_ts-dropdown',
                                                                value = 'voltage',
                                                                options = options_fields),
                                                dcc.Dropdown(id = 'resampling_ts-dropdown',
                                                                value = '1D',
                                                                options = option_resampling),
                                                dcc.Graph(id = 'ts-graph', figure = {}),   
                                        ], width = 5), #close Column 1 (Row 2)
                                        #Column 2 (right): Dropdown list + statistic graph
                                        dbc.Col([dcc.Dropdown(id = 'stat-dropdown', multi = False, value = 'Jan', options = [{'label':x, 'value': x} for x in list_months]),
                                                dcc.Graph(id = 'stat-graph', figure = {})      
                                        ], width = 5), #close Column 2 (Row 2)
                                ]), # Close Row #2

                            dbc.Row([
                                    dcc.Dropdown(id = 'field_profile-dropdown', multi = False, value = 'net_power_battery', options = options_fields),
                                    dcc.Dropdown(id = 'n_batteries_profile-dropdown', multi = False, value = 5, options = options_n_random_batteries),
                                    dcc.Graph(id = 'profile-graph', figure = {})      
                                     ]),

                            #Row 3: Cluster results visualization
                            dbc.Row([
                                     #Title of the section
                                     dbc.Col([html.H1('Cluster results visualization', className = 'text-left mb-4')], width = 12),
                                     #Column 1 (left): Dropdown list + clusering results graph
                                     dbc.Col([dcc.Dropdown(id = 'cluster-dropdown', multi = False, value = 'Jan', options = [{'label':x, 'value': x} for x in list_months]),
                                              dcc.Graph(id = 'cluster-graph', figure = {})      
                                     ], width = 5), #close Column 1 (Row 3)
                                     #Column 2 (right): Dropdown list + statistic graph
                                     dbc.Col([dcc.Dropdown(id = 'stat_clust-dropdown', multi = False, value = 'Jan', options = [{'label':x, 'value': x} for x in list_months]),
                                              dcc.Graph(id = 'stat_clust-graph', figure = {})      
                                     ], width = 5), #close Column 2 (Row 3)
                            ]), # Close Row #3

                            #Row 4: Extracting user profiles
                            dbc.Row([
                                     #Title of the section
                                     dbc.Col([html.H1('Extract user profiles', className = 'text-left mb-4')], width = 12),
                                     #Column 1 (left): Dropdown list + K means graph
                                     dbc.Col([dcc.Dropdown(id = 'k-dropdown', multi = False, value = 'Jan', options = [{'label':x, 'value': x} for x in list_months]),
                                              dcc.Graph(id = 'k-graph', figure = {})      
                                     ], width = 7), #close Column 1 (Row 4)
                                     #Column 2 (right): Score card placeholder
                                     dbc.Col([html.P('Score card placeholder', className = 'text-left mb-4')], width = 3),
                            ])
], fluid = True) #Close the whole dbc container. 

In [11]:
# data prep
# raw data as input
@app.callback(
    Output(component_id = 'profile-graph', component_property = 'figure'),
    [
    Input(component_id = 'field_profile-dropdown', component_property = 'value'),
    Input(component_id = 'n_batteries_profile-dropdown', component_property = 'value'),
    ]
)
def callback_plot_median_profile(field, n_batteries):
    return median_profiles.plot(field, n_batteries, df)

@app.callback(
    Output(component_id = 'ts-graph', component_property = 'figure'),
    [
    Input(component_id = 'n_batteries_ts-dropdown', component_property = 'value'),
    Input(component_id = 'var_ts-dropdown', component_property = 'value'),
    Input(component_id = 'resampling_ts-dropdown', component_property = 'value'),
    ]
)
def callback_plot_time_series(n_batteries, selected_var, resampling):
    return plot_times_series.plot(n_batteries, selected_var, resampling, df)

In [12]:
if __name__ == '__main__':
  app.run_server(debug = True)

Dash app running on http://127.0.0.1:8050/


C:\Users\PC_victor\Anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

